In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
df_train = pd.DataFrame(train)
df_train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
df_test = pd.DataFrame(test)
df_test.head()

In [ ]:
X = df_train.drop(['Id','SalePrice'],axis=1)

In [ ]:
y = df_train['SalePrice']

In [ ]:
test_index = test['Id']
df_test.drop('Id',axis=1,inplace=True)

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.shape

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
cat_feats = [col for col in df_train.columns if df_train[col].dtype == 'object']
num_feats = [col for col in df_train.columns if df_train[col].dtype in ['float64', 'int64']]

In [ ]:
df_train[cat_feats].nunique()

In [ ]:
df_train[cat_feats].isnull().sum()

In [ ]:
for col in df_train[cat_feats].columns:
    sns.barplot(x=df_train[cat_feats][col].value_counts().index, y=df_train[cat_feats][col].value_counts())
    plt.title(col)
    plt.show()

In [ ]:
df_train[num_feats].nunique()

In [ ]:
df_train[num_feats].isnull().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
sns.heatmap(df_train[num_feats].corr())

In [ ]:
for col in df_train[num_feats].columns:
    plt.hist(df_train[num_feats][col])
    plt.title(col)
    plt.show()

In [ ]:
num_col = list(df_train[['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
'MiscVal', 'MoSold', 'YrSold']])

In [ ]:
cat_col = list(df_train[['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition']])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [ ]:
y

In [ ]:
df_train.head()

In [ ]:
train

In [ ]:
num_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                ('StandardScaler', StandardScaler())])
cat_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                ('OneHot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer(transformers=[('num', num_transform, num_col),
                                            ('cat', cat_transform, cat_col)])

In [ ]:
model = XGBRegressor(n_estimators=1000,random_state=0,learning_rate=0.05)

In [ ]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

In [ ]:
my_pipeline.fit(X, y)

In [ ]:
predictions = my_pipeline.predict(df_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
score = mean_squared_error(predictions, y)
print(score)

In [ ]:
scores = cross_val_score(my_pipeline, X, y, cv=5)
print(np.mean(scores))